# 装饰器

了解几个问题：
1. 装饰器是什么
2. 有什么用途，有什么好处
3. 类似的设计程序设计理念
4. 怎么实现装饰器
5. 系统自带的装饰器

## 装饰器是什么

闭包

Python 的一个语法糖

## 用途及好处

更加灵活地添加新的功能，使代码更简洁，可读性更强；

数据校验 参考 httpauth 中的 verify_password

事务处理

缓存

日志

监控

调试

业务规则

压缩

加密

## 类似的设计理念

设计模式中的装饰器模式，但千万别和设计模式里面的装饰器模式搞混了，都是想要对一个已有的模块做一些“装饰工作”，所谓装饰工作就是想**给现有的模块加上一些小装饰（一些小功能，这些小功能可能好多模块都会用到），但又不让这个小装饰（小功能）侵入到原有的模块中的代码里去**。

Python 的 Decorator在使用上和Java/C#的Annotation很相似，就是在方法名前面加一个@XXX注解来为这个方法装饰一些东西。

无论何时我们想对一个对象添加额外的功能，都有下面这些不同的可选方法。

如果合理，可以直接将功能添加到对象所属的类（例如，添加一个新的方法）
使用组合
使用继承

设计模式为我们提供第四种可选方法，以支持动态地（运行时）扩展一个对象的功能，这种方法就是装饰器。装饰器（Decorator）模式能够以透明的方式（不会影响其他对象）动态地将功能添加到一个对象中

## 实现方法

In [3]:
def hello(fn):
    def wrapper():
        print("hello, %s" % fn.__name__)
        fn()
        print("goodby, %s" % fn.__name__)
    return wrapper
 
@hello
def foo():
    print("i am foo")
 
foo()

hello, foo
i am foo
goodby, foo


In [28]:
def memo(func):
    """
    remember
    """
    d = dict()
    def wrapper(x):
        if x not in d:            
            d[x] = func(x)
        return d[x]
    return wrapper

@memo
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

fib(55)

139583862445

In [25]:
class memorize:
    """
    remember
    """
    def __init__(self, func):
        self.func = func
        self.d = {}
        
    def __call__(self, *args):
        if args not in self.d:
            self.d[args] = self.func(*args)
        return self.d[args]
    
@memorize
def fibc(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibc(n-1) + fibc(n-2)

fibc(55)

139583862445

In [41]:
import time
# from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
#     @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [39]:
import time
from functools import wraps
def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [40]:
@timethis
def countdown(n):
    '''
    Counts down
    '''
    while n > 0:
        n -= 1
    print('done')
countdown(100000)

done
countdown 0.00716710090637207


产生了一个问题：

In [41]:
countdown.__name__

'countdown'

In [43]:
countdown.__doc__

'\n    Counts down\n    '

In [49]:
countdown.__annotations__

{}

可以看到，使用 functools的 wraps 装饰器，可以保留装饰器函数中的 meta 信息。

In [55]:
countdown.__wrapped__(1000000)  #直接访问被包装函数

done


In [45]:
def memo(func):
    """
    remember
    """
    d = dict()
    def wrapper(x):
        if x not in d:            
            d[x] = func(x)
        return d[x]
    return wrapper

@memo
@timethis
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

fib(55)

fib 9.5367431640625e-07
fib 7.152557373046875e-07
fib 0.0006048679351806641
fib 0.0010538101196289062
fib 0.0013806819915771484
fib 0.0014410018920898438
fib 0.0014939308166503906
fib 0.0015439987182617188
fib 0.0015947818756103516
fib 0.001631021499633789
fib 0.0016620159149169922
fib 0.0016970634460449219
fib 0.0017719268798828125
fib 0.0018112659454345703
fib 0.0018470287322998047
fib 0.0018832683563232422
fib 0.0019180774688720703
fib 0.0026938915252685547
fib 0.0027709007263183594
fib 0.002835988998413086
fib 0.0028989315032958984
fib 0.0029609203338623047
fib 0.0030269622802734375
fib 0.003087282180786133
fib 0.0031502246856689453
fib 0.004269123077392578
fib 0.004378080368041992
fib 0.004435062408447266
fib 0.0044901371002197266
fib 0.004547119140625
fib 0.004600048065185547
fib 0.004654884338378906
fib 0.004726886749267578
fib 0.004773855209350586
fib 0.004832029342651367
fib 0.004892826080322266
fib 0.0049326419830322266
fib 0.004973173141479492
fib 0.005011081695556641
fib 0.

139583862445

In [48]:
memo(timethis(fib))(55)

wrapper 9.5367431640625e-07


139583862445

如果想要给装饰器传递参数，丰富装饰器的行为：

In [35]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!!!')    
    
add(3,4)


7

In [32]:
spam()

Spam!!!


In [36]:
foo()

happy happy


#### 协助调试

In [16]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)

    return wrapper

In [20]:
@optional_debug
def spam(a,b,c):
    print(a, b, c)

spam(3,4,5)


3 4 5


In [21]:
spam(3,4,5,debug=True)

Calling spam
3 4 5


由此可见，很多们在使用 web 框架的时候，看到的各种各样的装饰器语法，都是可以实现的：

In [17]:
def makeHtmlTag(tag, *args, **kwds):
    def real_decorator(fn):
        css_class = " class='{0}'".format(kwds["css_class"]) \
                                     if "css_class" in kwds else ""
        def wrapped(*args, **kwds):
            return "<"+tag+css_class+">" + fn(*args, **kwds) + "</"+tag+">"
        return wrapped
    return real_decorator
 
@makeHtmlTag(tag="b", css_class="bold_css")
@makeHtmlTag(tag="i", css_class="italic_css")
def hello():
    return "hello world"
 
hello()
 
# 输出：
# <b class='bold_css'><i class='italic_css'>hello world</i></b>

"<b class='bold_css'><i class='italic_css'>hello world</i></b>"

In [14]:
class MyApp():
    def __init__(self):
        self.func_map = {}
 
    def register(self, name):
        def func_wrapper(func):
            self.func_map[name] = func
            return func
        return func_wrapper
 
    def call_method(self, name=None):
        func = self.func_map.get(name, None)
        if func is None:
            raise Exception("No function registered against - " + str(name))
        return func()
 
app = MyApp()
 
@app.register('/')
def main_page_func():
    return "This is the main page."
 
@app.register('/next_page')
def next_page_func():
    return "This is the next page."

至此，可以得到两种装饰器的模板，不是太复杂的装饰器都可以直接套用：

In [1]:
def decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):

        # 逻辑 
            
        return func(*args, **kwargs)
    return wrapper


In [3]:
def myDecorator(...):　　　　#定义装饰器，可能带参数
     def decorator(func):    #装饰器核心，以被装饰的函数对象为参数，返回装饰后的函数对象
         def wrapper(*args, **kwargs):    #装饰的过程，参数列表适应不同参数的函数
             ...    #修改函数调用前的行为
             func(*args, **kvargs)    #调用函数
             ...    #修改函数调用后的行为
         return wrapper
     return decorator
 
@myDecorator(...):　　　　#给函数加上装饰器
def myFunc(...):　　　　  #自己定义的功能函数
    ...

## 系统自带的装饰器

In [4]:
from functools import lru_cache

@lru_cache(maxsize=32)
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
    
fib(55)

139583862445

In [6]:
from contextlib import contextmanager

@contextmanager
def make_context():
    print('  entering')
    try:
        yield {}
    except RuntimeError as err:
        print('  ERROR:', err)
    finally:
        print('  exiting')

print('Normal:')
with make_context() as value:
    print('  inside with statement:', value)

print('\nHandled error:')
with make_context() as value:
    raise RuntimeError('showing example of handling an error')

print('\nUnhandled error:')
with make_context() as value:
    raise ValueError('this exception is not handled')

Normal:
  entering
  inside with statement: {}
  exiting

Handled error:
  entering
  ERROR: showing example of handling an error
  exiting

Unhandled error:
  entering
  exiting


ValueError: this exception is not handled

In [8]:
from functools import singledispatch


@singledispatch
def myfunc(arg):
    print('default myfunc({!r})'.format(arg))


@myfunc.register(int)
def myfunc_int(arg):
    print('myfunc_int({})'.format(arg))


@myfunc.register(list)
def myfunc_list(arg):
    print('myfunc_list()')
    for item in arg:
        print('  {}'.format(item))

myfunc('string argument')
myfunc(1)
myfunc(2.3)
myfunc(['a', 'b', 'c'])

default myfunc('string argument')
myfunc_int(1)
default myfunc(2.3)
myfunc_list()
  a
  b
  c


参考文档：

[python decorator library](https://wiki.python.org/moin/PythonDecoratorLibrary)

[python cookbook](https://python3-cookbook.readthedocs.io/zh_CN/latest/)

[coolshell](https://coolshell.cn/articles/11265.html)

[流畅的 Python]
